In [1]:
### mount on google drive
from google.colab import drive
drive.mount('/content/drive/')
# go to your code directory
import os
os.chdir("/content/drive/My Drive/Earth-surface-water-mapping")
# !ls
# !nvidia-smi


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import numpy as np
import time
import random
from utils import acc_patch
from data_loader import get_scene, get_patch
from models import MobileNetV2, DeepLabV3Plus_improve


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
## training dataset (!!your dataset directory)
folder_TrainScenes = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/TrainingScene/'
folder_TrainTruths = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/TrainingTruth/'
## super parameters
PATCH_SIZE = 512
EPOCHS = 500
lr = 0.005
BATCH_SIZE = 4
BUFFER_SIZE = 200

In [4]:
# get training data
Scenes, Truths = get_scene(folder_TrainScenes, folder_TrainTruths, PATCH_SIZE)
TrainSet = get_patch(Scenes, Truths, PATCH_SIZE, BATCH_SIZE, BUFFER_SIZE)
## model configuration
model = DeepLabV3Plus_improve(input_shape=(PATCH_SIZE, PATCH_SIZE, 6), base_model=MobileNetV2,  
                      d_feature=91, m_feature=24, l_feature=11, nclasses=2)
model_loss = tf.keras.losses.BinaryCrossentropy()

*** Building DeepLabv3Plus Network ***
*** Output_Shape => (None, 512, 512, 1) ***


In [5]:
def train_step(TrainSet, lr):
    TraAcc_OA_epoch = []
    TraAcc_MIoU_epoch = []
    Train_loss_epoch = []
    model_optimizer = tf.keras.optimizers.Adam(lr)
    for Train_Patch, Train_Truth in TrainSet:
        with tf.GradientTape() as tape:
            outp_TraPatch = model(Train_Patch, training=True)  
            outp_loss = model_loss(Train_Truth, outp_TraPatch)
            model_gradients = tape.gradient(outp_loss, model.trainable_variables)
            model_optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))
            Train_loss_epoch.append(outp_loss)
            TraAcc_OA, TraAcc_MIoU = acc_patch(Train_Truth, outp_TraPatch)
            TraAcc_OA_epoch.append(TraAcc_OA)
            TraAcc_MIoU_epoch.append(TraAcc_MIoU)
    Train_loss = np.mean(np.array(Train_loss_epoch)) 
    TraAcc_OA = np.mean(np.array(TraAcc_OA_epoch))
    TraAcc_MIoU = np.mean(np.array(TraAcc_MIoU_epoch))
    return Train_loss, TraAcc_OA, TraAcc_MIoU

def train_epoch(EPOCHS):
    Train_loss_plot = []
    TraAcc_OA_plot = []
    TraAcc_MIoU_plot = []
    start = time.time()
    for epoch in range(EPOCHS): 
        lr_new = lr*((1-epoch/EPOCHS)**0.9)
        Train_loss_step, TraAcc_OA_step, TraAcc_MIoU_step = train_step(TrainSet, lr_new)
        print('Epoch: {}, Train loss: {}, Train OA: {}, Train MIoU: {}, Time taken: {} sec'.format(
            epoch + 1, Train_loss_step, TraAcc_OA_step, TraAcc_MIoU_step, time.time() - start))
        Train_loss_plot.append(Train_loss_step)
        TraAcc_OA_plot.append(TraAcc_OA_step)
        TraAcc_MIoU_plot.append(TraAcc_MIoU_step)
    return Train_loss_plot, TraAcc_OA_plot, TraAcc_MIoU_plot


In [ ]:
####  model training and save the trained model (!!change to the user's directory)
train_epoch(EPOCHS)
# model.save('/content/drive/My Drive/Earth-surface-water-mapping/pretrained_model/pretrained_model_v1')